In [1]:
import pandas as pd 
import numpy as np
import datetime
import math
import matplotlib.pyplot as plt #default library for making plots
import seaborn as sns           #use to make nicer plots
import matplotlib

In [2]:
#mod_data is Adele's modified data --> not raw data downloaded from fitbit
#data from multiple sources have been copied and pasted into an Excel sheet

mod_data = pd.read_csv("All Fitbit Data.csv") 
mod_data.head()

,Activities,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
1,9/1/17,"2,520","18,402",7.79,59,438,301,107,27,"1,553"
2,9/2/17,"2,857","27,273",11.62,70,644,355,111,48,"1,954"
3,9/3/17,"2,212","15,794",6.68,16,657,249,76,4,"1,151"
4,9/4/17,"2,843","28,438",12.1,125,475,351,100,55,"1,951"


In [3]:
import datetime
import calendar

def day_of_week(unmodified_date): 
    date = unmodified_date.split('/')  #split unmodified date into a list of strings
    
    date[2] = '20' + date[2]    #fixes date syntax in order to use datetime function
    if len(date[0]) == 1:       #changes dates to mm-dd-yyyy format in order to use .weekday()
        date[0] = '0' + date[0] 
    if len(date[1]) == 1:
        date[1] = '0' + date[1]
    date = '-'.join(date)
    
    return datetime.datetime.strptime(date, '%m-%d-%Y').weekday() # returns day of week as a number
           

In [4]:
#relabeled the column names
mod_data = mod_data.rename(columns=dict(zip(mod_data.columns, mod_data.loc[0, :].tolist()))).drop(0)
mod_data['Day of Week'] = mod_data['Date'].apply(day_of_week)

In [5]:
mod_data.head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,Day of Week
1,9/1/17,"2,520","18,402",7.79,59,438,301,107,27,"1,553",4
2,9/2/17,"2,857","27,273",11.62,70,644,355,111,48,"1,954",5
3,9/3/17,"2,212","15,794",6.68,16,657,249,76,4,"1,151",6
4,9/4/17,"2,843","28,438",12.1,125,475,351,100,55,"1,951",0
5,9/5/17,"1,980","12,233",5.18,34,720,160,37,36,843,1


In [6]:
#How do we extract and combine the data from multiple json files?

In [7]:
#Example of how to load one of the json files onto jupyter notebook.

import json

with open("json_test_files/heart_rate-2017-05-30.json", 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
df.head()

,dateTime,value
0,05/30/17 17:49:31,"{'bpm': 70, 'confidence': 0}"
1,05/30/17 17:49:41,"{'bpm': 110, 'confidence': 0}"
2,05/30/17 17:49:46,"{'bpm': 114, 'confidence': 0}"
3,05/30/17 17:49:51,"{'bpm': 110, 'confidence': 0}"
4,05/30/17 17:50:00,"{'bpm': 110, 'confidence': 0}"


In [8]:
import os

# path_to_json = 'json_test_files/'    #need to figure out how to find name of folder in the future
path_to_json = 'Adele_files/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')\
              and not pos_json.startswith('badge')]
json_files

['time_in_heart_rate_zones-2018-06-06.json',
 'time_in_heart_rate_zones-2017-10-05.json',
 'time_in_heart_rate_zones-2018-08-18.json',
 'heart_rate-2017-06-19.json',
 'heart_rate-2018-12-27.json',
 'time_in_heart_rate_zones-2019-02-01.json',
 'distance-2018-12-20.json',
 'heart_rate-2018-02-27.json',
 'heart_rate-2017-08-07.json',
 'demographic_vo2_max-2018-12-30.json',
 'time_in_heart_rate_zones-2017-11-28.json',
 'time_in_heart_rate_zones-2018-05-16.json',
 'time_in_heart_rate_zones-2019-01-11.json',
 'altitude-2019-02-18.json',
 'time_in_heart_rate_zones-2018-02-12.json',
 'time_in_heart_rate_zones-2018-12-12.json',
 'steps-2018-03-25.json',
 'sedentary_minutes-2018-04-24.json',
 'heart_rate-2017-10-30.json',
 'time_in_heart_rate_zones-2018-01-02.json',
 'heart_rate-2019-03-19.json',
 'heart_rate-2019-01-24.json',
 'time_in_heart_rate_zones-2018-11-02.json',
 'time_in_heart_rate_zones-2017-07-01.json',
 'moderately_active_minutes-2019-02-18.json',
 'heart_rate-2018-05-23.json',
 'di

In [9]:
#Extracts first three objects of a string.
def first_3_letters(string):
    return string[0:3]

#Takes in a list of strings, extracts the first three objects of each string, and returns a list of unique values from
#the extracted values.
def unique_first_3(alist): 
    result = []
    for i in alist:
        result.append(first_3_letters(i))
    return list(set(result))

#Takes in a list of filenames and groups them according to the first three letters of the filename.
def group_files(list_files):
    starts = unique_first_3(list_files)
    copy_json_files = list_files
    
    grouped_files = [[] for i in range(len(starts))]
    for i in range(len(starts)):
        for j in range(len(list_files)):
            if (starts[i] == first_3_letters(copy_json_files[j])):
                grouped_files[i].append(copy_json_files[j])
    return grouped_files

In [10]:
json_files

['time_in_heart_rate_zones-2018-06-06.json',
 'time_in_heart_rate_zones-2017-10-05.json',
 'time_in_heart_rate_zones-2018-08-18.json',
 'heart_rate-2017-06-19.json',
 'heart_rate-2018-12-27.json',
 'time_in_heart_rate_zones-2019-02-01.json',
 'distance-2018-12-20.json',
 'heart_rate-2018-02-27.json',
 'heart_rate-2017-08-07.json',
 'demographic_vo2_max-2018-12-30.json',
 'time_in_heart_rate_zones-2017-11-28.json',
 'time_in_heart_rate_zones-2018-05-16.json',
 'time_in_heart_rate_zones-2019-01-11.json',
 'altitude-2019-02-18.json',
 'time_in_heart_rate_zones-2018-02-12.json',
 'time_in_heart_rate_zones-2018-12-12.json',
 'steps-2018-03-25.json',
 'sedentary_minutes-2018-04-24.json',
 'heart_rate-2017-10-30.json',
 'time_in_heart_rate_zones-2018-01-02.json',
 'heart_rate-2019-03-19.json',
 'heart_rate-2019-01-24.json',
 'time_in_heart_rate_zones-2018-11-02.json',
 'time_in_heart_rate_zones-2017-07-01.json',
 'moderately_active_minutes-2019-02-18.json',
 'heart_rate-2018-05-23.json',
 'di

In [11]:
grouped = group_files(json_files)
grouped

[['time_in_heart_rate_zones-2018-06-06.json',
  'time_in_heart_rate_zones-2017-10-05.json',
  'time_in_heart_rate_zones-2018-08-18.json',
  'time_in_heart_rate_zones-2019-02-01.json',
  'time_in_heart_rate_zones-2017-11-28.json',
  'time_in_heart_rate_zones-2018-05-16.json',
  'time_in_heart_rate_zones-2019-01-11.json',
  'time_in_heart_rate_zones-2018-02-12.json',
  'time_in_heart_rate_zones-2018-12-12.json',
  'time_in_heart_rate_zones-2018-01-02.json',
  'time_in_heart_rate_zones-2018-11-02.json',
  'time_in_heart_rate_zones-2017-07-01.json',
  'time_in_heart_rate_zones-2018-12-04.json',
  'time_in_heart_rate_zones-2017-08-24.json',
  'time_in_heart_rate_zones-2018-02-04.json',
  'time_in_heart_rate_zones-2017-07-17.json',
  'time_in_heart_rate_zones-2018-11-14.json',
  'time_in_heart_rate_zones-2018-03-29.json',
  'time_in_heart_rate_zones-2018-01-14.json',
  'time_in_heart_rate_zones-2017-09-09.json',
  'time_in_heart_rate_zones-2017-10-13.json',
  'time_in_heart_rate_zones-2018-0

In [12]:
path_to_json

'Adele_files/'

In [14]:
#path_to_json is hardcoded to the folder name. Will need to address this issue later so that we can upload the 
#fitbit data from any folder regardless of folder name.


#Takes in a list of lists where the files are grouped according to its content. For instance, each nested list will 
#have the name of files that have the same column names. Merges the files within each nested list. Returns the merged
#files as dataframes that are stored within a list.

def form_dataframes(grouped_list):
    dataframes = [[] for i in range(len(grouped_list))]
    final_dfs = []      
    
    #converts the filenames to dataframes
    for i in range(len(grouped_list)):
        with open(path_to_json + grouped_list[i][0], 'r') as first:
            first_filename = json.load(first)
            df1 = pd.DataFrame(first_filename)
            dataframes[i].append(df1)
        for j in range(1, len(grouped_list[i])):
            with open(path_to_json + grouped_list[i][j], 'r') as f:
                data = json.load(f)
                df = pd.DataFrame(data)
                dataframes[i].append(df)
    
    #combines all similar dataframes
    for i in range(len(dataframes)):
        final_dfs.append(pd.concat(dataframes[i], sort=True))
    
    return final_dfs

In [33]:
#list of all dataframes
all_dfs = form_dataframes(grouped)

In [34]:
test = all_dfs[0]
date1 = test.iloc[0,0]
date1

'06/06/18 00:00:00'

In [35]:
#separate
test.head()

,dateTime,value
0,06/06/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 120...
0,10/05/17 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 54.0, ..."
0,08/18/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 110...
0,02/01/19 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_2': 0.0, '..."
0,11/28/17 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 80.0, ..."


In [56]:
#input is a date as a string. returns a datetime object
def separate_datetime(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S')

#returns day of week as integer for reference --> {0: Sunday,...., 6:Saturday}
def weekday(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').weekday()

#inputs day of week as integer. 
def is_weekend(day):
    if day == 0 or day == 6:
        return True
    else:
        return False

def month(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').month

def day_of_month(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').day
    
def year(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').year
    
def hour(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').hour
    
def minute(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').minute

def second(date):
    return datetime.datetime.strptime(date, '%m/%d/%y %H:%M:%S').second



#input dataframe with 'dateTime' column, adds date features to dataframe
def add_date_features(dataframe):
    
    series = dataframe['dateTime']
        
    dataframe['Day of Week'] = series.apply(weekday)
    dataframe['Is Weekend'] = dataframe['Day of Week'].apply(is_weekend)
    dataframe['Month'] = series.apply(month)
    dataframe['Day'] = series.apply(day_of_month)
    dataframe['Year'] = series.apply(year)
    dataframe['Hour'] = series.apply(hour)
    dataframe['Minute'] = series.apply(minute)
    dataframe['Second'] = series.apply(second)
    return dataframe

In [57]:
add_date_features(test)

,dateTime,value,Day of Week,Is Weekend,Month,Day,Year,Hour,Minute,Second
0,06/06/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 120...,2,False,6,6,2018,0,0,0
0,10/05/17 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 54.0, ...",3,False,10,5,2017,0,0,0
0,08/18/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 110...,5,False,8,18,2018,0,0,0
0,02/01/19 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_2': 0.0, '...",4,False,2,1,2019,0,0,0
0,11/28/17 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 80.0, ...",1,False,11,28,2017,0,0,0
0,05/16/18 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_2': 30.0, ...",2,False,5,16,2018,0,0,0
0,01/11/19 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 126...,4,False,1,11,2019,0,0,0
0,02/12/18 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_3': 2.0, '...",0,True,2,12,2018,0,0,0
0,12/12/18 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_3': 0.0, '...",2,False,12,12,2018,0,0,0
0,01/02/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 109...,1,False,1,2,2018,0,0,0


In [48]:
test['dateTime'].apply(separate_datetime).reset_index().drop(columns=['index']).iloc[0,0]

Timestamp('2018-06-06 00:00:00')

In [16]:
all_dfs[0].head(5)

,dateTime,value
0,06/19/17 04:00:01,"{'bpm': 55, 'confidence': 3}"
1,06/19/17 04:00:06,"{'bpm': 56, 'confidence': 3}"
2,06/19/17 04:00:21,"{'bpm': 59, 'confidence': 3}"
3,06/19/17 04:00:36,"{'bpm': 59, 'confidence': 3}"
4,06/19/17 04:00:51,"{'bpm': 61, 'confidence': 3}"


In [17]:
all_dfs[1].head(5)

,dateTime,value
0,12/31/16 00:00:00,1650


In [18]:
all_dfs[2].head(5)

,activeDuration,activityLevel,activityName,activityTypeId,averageHeartRate,calories,distance,distanceUnit,duration,elevationGain,...,originalDuration,originalStartTime,pace,shouldFetchDetails,source,speed,startTime,steps,tcxLink,vo2Max
0,1843000,"[{'minutes': 0, 'name': 'sedentary'}, {'minute...",Walk,90013,103.0,123,NaN,NaN,1843000,0.0,...,1843000,06/11/18 18:15:57,NaN,False,NaN,NaN,06/11/18 18:15:57,2190.0,NaN,NaN
1,3942000,"[{'minutes': 5, 'name': 'sedentary'}, {'minute...",Walk,90013,102.0,287,NaN,NaN,3942000,20.0,...,3942000,06/11/18 19:29:20,NaN,False,NaN,NaN,06/11/18 19:29:20,5299.0,NaN,NaN
2,3072000,"[{'minutes': 5, 'name': 'sedentary'}, {'minute...",Walk,90013,100.0,184,NaN,NaN,3072000,0.0,...,3072000,06/12/18 08:42:05,NaN,False,NaN,NaN,06/12/18 08:42:05,3022.0,NaN,NaN
3,2304000,"[{'minutes': 0, 'name': 'sedentary'}, {'minute...",Walk,90013,110.0,165,NaN,NaN,2304000,10.0,...,2304000,06/12/18 11:48:07,NaN,False,NaN,NaN,06/12/18 11:48:07,2931.0,NaN,NaN
4,2151000,"[{'minutes': 1, 'name': 'sedentary'}, {'minute...",Walk,90013,105.0,148,NaN,NaN,2151000,50.0,...,2151000,06/12/18 12:58:05,NaN,False,NaN,NaN,06/12/18 12:58:05,2585.0,NaN,NaN


In [19]:
all_dfs[3].head(5)

,dateTime,value
0,05/31/17 17:02:39,"{'exerciseId': 7994735916, 'runVO2Max': 36.914..."
1,06/01/17 16:09:10,"{'exerciseId': 8011597564, 'runVO2Max': 44.166..."
2,06/03/17 15:17:25,"{'exerciseId': 8050497515, 'runVO2Max': 41.221..."
3,06/04/17 14:02:24,"{'exerciseId': 8067986438, 'runVO2Max': 45.485..."


In [20]:
all_dfs[4].head(5)

,dateTime,value
0,12/20/18 08:00:00,0
1,12/20/18 08:01:00,0
2,12/20/18 08:02:00,0
3,12/20/18 08:03:00,0
4,12/20/18 08:04:00,0


In [21]:
all_dfs[5].head(5)

,dateTime,value
0,03/25/18 07:00:00,0
1,03/25/18 07:01:00,0
2,03/25/18 07:02:00,0
3,03/25/18 07:03:00,0
4,03/25/18 07:04:00,0


In [22]:
all_dfs[6].head(5)

,bmi,date,logId,source,time,weight
0,19.83,12/31/16,1483228799000,API,23:59:59,118.9


In [23]:
all_dfs[7].head(5)

,dateTime,value
0,12/20/18 00:00:00,226
1,12/21/18 00:00:00,240
2,12/22/18 00:00:00,317
3,12/23/18 00:00:00,277
4,12/24/18 00:00:00,377


In [24]:
all_dfs[8].head(5)

,dateTime,value
0,12/30/18 00:00:00,"{'demographicVO2Max': 40.60488, 'demographicVO..."
1,12/31/18 00:00:00,"{'demographicVO2Max': 40.14795, 'demographicVO..."
2,01/01/19 00:00:00,"{'demographicVO2Max': 39.344620000000006, 'dem..."
3,01/02/19 00:00:00,"{'demographicVO2Max': 39.24459, 'demographicVO..."
4,01/03/19 00:00:00,"{'demographicVO2Max': 40.299080000000004, 'dem..."


In [25]:
all_dfs[9].head(5)

,dateOfSleep,duration,efficiency,endTime,infoCode,levels,logId,minutesAfterWakeup,minutesAsleep,minutesAwake,minutesToFallAsleep,startTime,timeInBed,type
0,2017-12-25,29820000,97,2017-12-25T06:26:00.000,0,"{'summary': {'deep': {'count': 5, 'minutes': 7...",16683423534,0,442,55,0,2017-12-24T22:08:30.000,497,stages
1,2017-12-24,3900000,88,2017-12-24T18:42:30.000,2,"{'summary': {'restless': {'count': 3, 'minutes...",16683423533,0,57,8,0,2017-12-24T17:37:00.000,65,classic
2,2017-12-24,24480000,95,2017-12-24T06:55:30.000,0,"{'summary': {'deep': {'count': 3, 'minutes': 4...",16669911385,0,357,51,0,2017-12-24T00:07:00.000,408,stages
3,2017-12-23,7980000,96,2017-12-23T16:28:00.000,2,"{'summary': {'restless': {'count': 2, 'minutes...",16666065337,0,128,5,0,2017-12-23T14:15:00.000,133,classic
4,2017-12-22,32400000,94,2017-12-22T20:25:30.000,0,"{'summary': {'deep': {'count': 7, 'minutes': 8...",16661798489,0,448,92,0,2017-12-22T11:25:30.000,540,stages


In [26]:
all_dfs[10].head(5)

,dateTime,value
0,02/28/17 00:00:00,0.92
1,02/28/17 00:01:00,0.92
2,02/28/17 00:02:00,2.2
3,02/28/17 00:03:00,0.92
4,02/28/17 00:04:00,0.92


In [27]:
all_dfs[11].head(5)

,dateTime,value
0,06/28/17 00:00:00,94
1,06/29/17 00:00:00,37
2,06/30/17 00:00:00,15
3,07/01/17 00:00:00,98
4,07/02/17 00:00:00,26


In [28]:
all_dfs[12].head(5)

,dateTime,value
0,04/24/18 00:00:00,603
1,04/25/18 00:00:00,568
2,04/26/18 00:00:00,1049
3,04/27/18 00:00:00,566
4,04/28/18 00:00:00,590


In [29]:
all_dfs[13].head(5)

,dateTime,value
0,12/30/16 00:00:00,"{'date': None, 'value': 0.0, 'error': 0.0}"
1,12/31/16 00:00:00,"{'date': None, 'value': 0.0, 'error': 0.0}"
2,01/01/17 00:00:00,"{'date': None, 'value': 0.0, 'error': 0.0}"
3,01/02/17 00:00:00,"{'date': None, 'value': 0.0, 'error': 0.0}"
4,01/03/17 00:00:00,"{'date': None, 'value': 0.0, 'error': 0.0}"


In [30]:
all_dfs[14].head(5)

,dateTime,value
0,02/18/19 00:31:00,20
1,02/18/19 00:36:00,10
2,02/18/19 11:31:00,10
3,02/18/19 11:38:00,10
4,02/18/19 12:38:00,10


In [31]:
all_dfs[15].head(5)

,dateTime,value
0,02/18/19 00:00:00,28
1,02/19/19 00:00:00,53
2,02/20/19 00:00:00,52
3,02/21/19 00:00:00,36
4,02/22/19 00:00:00,37


In [32]:
all_dfs[16].head(5)

,dateTime,value
0,06/06/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 120...
0,10/05/17 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 54.0, ..."
0,08/18/18 00:00:00,{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 110...
0,02/01/19 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_2': 0.0, '..."
0,11/28/17 00:00:00,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 80.0, ..."


In [33]:
'''1. need to create a fxn that separates dateTime
2. make a way that aranges nested lists alphabeticaly
3. extract values from tuples from certain tables (include way to relabel values)'''

'1. need to create a fxn that separates dateTime\n2. make a way that aranges nested lists alphabeticaly\n3. extract values from tuples from certain tables (include way to relabel values)'

In [34]:
# #for testing
# def date_to_day_of_week(date): 
#     born = datetime.datetime.strptime(date, '%m-%d-%Y').weekday() #returns day of week as a number
#     return calendar.day_name[born] <-- returns day of week as word
#for reference --> {0: Sunday,...., 6:Saturday}